[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/stable-diffusion-webui-colab/blob/drive/run.ipynb)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui

!python launch.py --share --xformers --enable-insecure-extension-access --gradio-queue --ui-config-file /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/ui-config.json --ui-settings-file /content/drive/MyDrive/stable-diffusion-webui-colab/stable-diffusion-webui/config.json